# 0. Import library

In [1]:
import sqlite3
import pandas as pd

import re
from typing import List, Dict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from difflib import SequenceMatcher

# for answer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# need library
# pip3 install torch torchvision 

# 1. Use  database file connect DB

## 1.1 database file

In [2]:
# database file
covid_file = "example-data/example-covid-vaccinations.sqlite3"
data_file="example-data/example-data.sqlite3"
simple_file="example-data/example-simple.sqlite3"
murder_file="example-data/sql-murder-mystery.sqlite3"
buildings_file="example-data/tallest_buildings_global.sqlite3"

database_files = [
    covid_file,
    data_file,
    simple_file,
    murder_file,
    buildings_file
]


## 1.2 connection

In [3]:
# build connection
covid_connection = sqlite3.connect(covid_file)
data_connection=sqlite3.connect(data_file)
simple_connection=sqlite3.connect(simple_file)
murder_connection=sqlite3.connect(murder_file)
buildings_connection=sqlite3.connect(buildings_file)

connections_list = [
    covid_connection,
    data_connection,
    simple_connection,
    murder_connection,
    buildings_connection
]

In [4]:
db=["covid","data","simple","murder","buildings"]

## 1.3 Generate 

 1. all_table_name=[]
 2. df_all_table
 3. table_db_mapping={}

In [5]:
df_all_table = {}
table_db_mapping={}
all_table_name=[]
i=0
for connection in connections_list:
    cursor = connection.cursor()
    
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    
    table_names = [name[0] for name in cursor.fetchall()]
    all_table_name.extend(table_names)
    
    for table_name in table_names:
 
        # 执行查询以从表格中检索数据，并将结果存储在 DataFrame 中
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, connection)
        
        table_db_mapping[table_name] = db[i];

        # 将 DataFrame 存储在字典中，以表格名称为键
        df_all_table[table_name] = df
    i=i+1

## 1.4 Test all the df are corect

In [6]:
all_table_name

['covid_vaccinations',
 'pre_ranking_filter_log',
 'pre_ranking_filter_key_mapping',
 'predicted_metric_log',
 'real_metric_log',
 'request_log',
 'users',
 'orders',
 'crime_scene_report',
 'drivers_license',
 'facebook_event_checkin',
 'interview',
 'get_fit_now_member',
 'get_fit_now_check_in',
 'solution',
 'income',
 'person',
 'talles_buildings']

In [7]:
table_db_mapping

{'covid_vaccinations': 'covid',
 'pre_ranking_filter_log': 'data',
 'pre_ranking_filter_key_mapping': 'data',
 'predicted_metric_log': 'data',
 'real_metric_log': 'data',
 'request_log': 'data',
 'users': 'simple',
 'orders': 'simple',
 'crime_scene_report': 'murder',
 'drivers_license': 'murder',
 'facebook_event_checkin': 'murder',
 'interview': 'murder',
 'get_fit_now_member': 'murder',
 'get_fit_now_check_in': 'murder',
 'solution': 'murder',
 'income': 'murder',
 'person': 'murder',
 'talles_buildings': 'buildings'}

In [8]:
# df_all_table['talles_buildings']

,rank,name,height_m,height_ft,year_built,floors_above,floors_below_ground,city,country
0,1,Burj Khalifa,828.0,2717.0,2010,163,1,Dubai,United Arab Emirates
1,2,Merdeka 118,678.9,2227.0,2022,118,5,Kuala Lumpur,Malaysia
2,3,Shanghai Tower,632.0,2073.0,2015,128,5,Shanghai,China
3,4,Abraj Al-Bait Clock Tower,601.0,1972.0,2012,120,3,Mecca,Saudi Arabia
4,5,Ping An International Finance Centre,599.1,1966.0,2017,115,5,Shenzhen,China
...,...,...,...,...,...,...,...,...,...
73,74,OKO Tower � South Tower,354.2,1162.0,2015,90,2,Moscow,Russia
74,75,The Marina Torch,352.0,1155.0,2011,86,4,Dubai,United Arab Emirates
75,76,Forum 66 Tower 1,350.6,1150.0,2015,68,4,Shenyang,China
76,77,The Pinnacle,350.3,1149.0,2012,60,6,Guangzhou,China


In [9]:
# df_all_table['pre_ranking_filter_log']

,filter_key,timestamp,task
0,o_template_id,2023-01-02 00:00:00,342111
1,o_blocking_publisher,2023-01-02 00:00:00,342111
2,o_score_rank,2023-01-02 00:00:00,342111
3,o_imprecise_diversity_advertiser,2023-01-02 00:00:00,342111
4,o_rank_filter_vector,2023-01-02 00:00:00,342111
...,...,...,...
5595,o_balance_low,2023-01-17 00:00:00,342117
5596,o_balance_low,2023-01-17 00:00:00,342117
5597,o_block_media,2023-01-17 00:00:00,342117
5598,o_imprecise_diversity_advertiser,2023-01-17 00:00:00,342117


# 2. Modeling


# 3. Function for get input data 
connect_fun(database_name: str) -> Object

that should “establish a connection” to a given SQLite file. That function will be run once before all the questions related to the given database.

## 3.1 connection function

In [10]:
def connect_fun(database_name: str) -> sqlite3.Connection:
    try:
        # connect db base on  database_name
        connection = sqlite3.connect(database_name)
        return connection
    except sqlite3.Error as e:
        print(f"Error connecting to {database_name}: {e}")
        return None

## 3.2 get table name base on connection

In [11]:
def get_table_names(connection):
    try:
        # Create a cursor object
        cursor = connection.cursor()

        # Execute a query to retrieve the names of all tables in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

        # Fetch the table names from the cursor results
        table_names = [name[0] for name in cursor.fetchall()]
        cursor.close()

        return table_names
    except sqlite3.Error as e:
        print(f"Error getting table names: {e}")
        return None

## 3.3 get column name base on table and connection

In [12]:
def get_columns_for_table(connection, table_name):
    column_names = []
    try:
        cursor = connection.cursor()

        # Execute PRAGMA query to get table column information, including column names
        cursor.execute(f"PRAGMA table_info({table_name});")

        # Fetch the results from the cursor, column names are in the first column (index 1)
        column_names = [row[1] for row in cursor.fetchall()]
        return column_names
        
        cursor.close()  # Close the cursor for this table
    except sqlite3.Error as e:
        print(f"Error getting column names for table {table_name}: {e}")
    
    return column_names

In [13]:
def get_columns_for_tables(connection, table_names):
    columns_dict = {}    
    try:
        cursor = connection.cursor()

        for table_name in table_names:
            # Execute PRAGMA query to get table column information, including column names
            cursor.execute(f"PRAGMA table_info({table_name});")

            # Fetch the results from the cursor, column names are in the first column (index 1)
            column_names = [row[1] for row in cursor.fetchall()]

            # Store the table name and its corresponding list of column names in the dictionary
            columns_dict[table_name] = column_names
            cursor.close()  # Close the cursor for this table
        return columns_dict
    except sqlite3.Error as e:
        print(f"Error getting column names: {e}")
        return None


## 3.4 unique value for each column

In [14]:
def get_unique_column_values(connection, table_name, column_name):

    try:
        # Create a cursor object
        cursor = connection.cursor()

        # Execute a query to retrieve the specified column's values
        cursor.execute(f"SELECT `{column_name}` FROM `{table_name}`;")

        # Fetch all values from the query result
        column_values = [row[0] for row in cursor.fetchall()]

        # Get unique values using set()
        unique_values = set(column_values)

        # Create a dictionary with column_name as the key
        unique_values_dict = {column_name: list(unique_values)}

        return unique_values_dict
    except sqlite3.Error as e:
        print(f"Error getting unique values: {e}")
        return {column_name: []}

## 3.5 Define class

In [15]:

class Column:
    def __init__(self, name, unique_values_dict=None):
        self.name = name
        self.unique_values = unique_values_dict if unique_values_dict is not None else {}

class Table:
    def __init__(self, name, columns=None):
        self.name = name
        self.columns = columns if columns is not None else []

    def add_column(self, column):

        self.columns.append(column)

## 3.6 get Table dictionary from connection

In [16]:
def get_tables_from_connection(connection):
    tables={}
    table_names=get_table_names(connection)
    
    for table_name in table_names:        
        column_names=get_columns_for_table(connection,table_name)
        
        columns=[]
        for column_name in column_names:
            
            unique_values=get_unique_column_values(connection, table_name, column_name)
            
            column_object=Column(column_name,unique_values)
            columns.append(column_object)
        table_object = Table(name=table_name,columns=columns)
        tables[table_name]=table_object 
    return tables

#### Test 1

In [17]:
conn=connect_fun("example-data/example-covid-vaccinations.sqlite3")
print(conn)

#### Test 2

In [18]:
table_names=get_table_names(conn)
table_names

['covid_vaccinations']

#### Test 3

In [19]:
print(table_names[0])

covid_vaccinations


In [20]:
c_list=get_columns_for_table(conn, table_names[0])
print(c_list)

['STATISTIC_CODE', 'Statistic_Label', 'TLIST(M1)', 'Month', 'C03898V04649', 'Local Electoral Area', 'C02076V03371', 'Age Group', 'UNIT', 'VALUE']


In [21]:
dict_column_names=get_columns_for_tables(conn, table_names)
dict_column_names

{'covid_vaccinations': ['STATISTIC_CODE',
  'Statistic_Label',
  'TLIST(M1)',
  'Month',
  'C03898V04649',
  'Local Electoral Area',
  'C02076V03371',
  'Age Group',
  'UNIT',
  'VALUE']}

#### test 4


In [22]:
get_unique_column_values(conn, "covid_vaccinations",dict_column_names["covid_vaccinations"][0] )

{'STATISTIC_CODE': ['CDC45C04',
  'CDC45C01',
  'CDC45C03',
  'CDC45C05',
  'CDC45C02',
  'CDC45C06']}

#### test 5

In [23]:
# Example usage:
database_file_path = "example-data/example-covid-vaccinations.sqlite3"

# Connect to the SQLite database
connection = sqlite3.connect(database_file_path)

# Specify the table name to build

tables=get_tables_from_connection(connection)

In [24]:
tables['covid_vaccinations'].name

'covid_vaccinations'

In [25]:
tables['covid_vaccinations'].columns[0].name

'STATISTIC_CODE'

In [26]:
tables['covid_vaccinations'].columns[0].unique_values

{'STATISTIC_CODE': ['CDC45C04',
  'CDC45C01',
  'CDC45C03',
  'CDC45C05',
  'CDC45C02',
  'CDC45C06']}

# 4. Modeling

## 4.1 Variable

In [27]:
# need library
# pip3 install torch torchvision 

In [28]:
tokenizer_one_table = AutoTokenizer.from_pretrained("juierror/text-to-sql-with-table-schema")
model_one_table = AutoModelForSeq2SeqLM.from_pretrained("juierror/text-to-sql-with-table-schema")


tokenizer = AutoTokenizer.from_pretrained("juierror/flan-t5-text2sql-with-schema-v2")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/flan-t5-text2sql-with-schema-v2")

## 4.1 Define the help function

In [29]:
def prepare_input_one_table(question: str, table_name: str, columns: List[str]):
    table_prefix = "table:"
    question_prefix = "question:"
    columns_prefix = "columns:"
    columns_str = ",".join(columns)
    inputs = f"{question_prefix} {question} {table_prefix} {table_name} {columns_prefix} {columns_str}"
    input_ids = tokenizer_one_table(inputs, max_length=700, return_tensors="pt").input_ids
    return input_ids

def fix_aggregate_syntax(sql_query, columns):
    # Define a regular expression pattern to match "MAX" or "MIN" followed by a column name
    pattern = r'(\bMAX\b|\bMIN\b)\s+([A-Za-z_][A-Za-z0-9_]*\b)'

    # Replace the matched pattern with the corrected syntax
    corrected_query = re.sub(pattern, r'\1(\2)', sql_query, flags=re.IGNORECASE)
    
    #replace with actual column names
    values_to_quote = columns
    
    for value in values_to_quote:
        pattern = fr'(?<=\s){re.escape(value)}(?=\s)'
        corrected_query = re.sub(pattern, f'"{value}"', corrected_query)
    
    # Check if the string ends with ";"
#     if not corrected_query.endswith(";"):
#         corrected_query += ";"
    
    return corrected_query


def get_prompt(tables, question):
    prompt = f"""convert question and table into SQL query. tables: {tables}. question: {question}"""
    return prompt

def prepare_input(question: str, tables: Dict[str, List[str]]):
    tables = [f"""{table_name}({",".join(tables[table_name])})""" for table_name in tables]
    tables = ", ".join(tables)
    prompt = get_prompt(tables, question)
    input_ids = tokenizer(prompt, max_length=512, return_tensors="pt").input_ids
    return input_ids

def contains_sensitive_word(query: str) -> bool:
    for word in ["password", "ssn", "private_key", "credit_card"]:
        if word in query.lower():  
            return True
    return False

## 4.3 Function for one table

In [30]:
def inference_one_table(question: str, table_name: str, columns: List[str], ) -> str:
    input_data = prepare_input_one_table(question=question, table_name=table_name, columns=columns)
    input_data = input_data.to(model_one_table.device)
    outputs = model_one_table.generate(inputs=input_data, num_beams=10, top_k=10, max_length=700)
    result = tokenizer_one_table.decode(token_ids=outputs[0], skip_special_tokens=True)

    result = fix_aggregate_syntax(result, columns)
    result = result.replace("table", table_name)
    
    # Check if "SELECT" and "FROM" are both present in the result
    if "SELECT" not in result or "FROM" not in result:
        return "The generated SQL query is incomplete or incorrect."
    
    # Check if the query tries to get any sensitive information
    if contains_sensitive_word(result):
        return "Your question requires sensitive information and is rejected."
    
    return result

## 4.4 Function for muti tables

In [31]:
def inference(question: str, tables: Dict[str, List[str]]) -> str:
    input_data = prepare_input(question=question, tables=tables)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=512)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    
    # adding all used columns to 1 variable
    all_tables_columns = [value for values_list in tables.values() for value in values_list]
    
    result=fix_aggregate_syntax(result, all_tables_columns)
    
    # Check if "SELECT" and "FROM" are both present in the result
    if "SELECT" not in result or "FROM" not in result:
        return "The generated SQL query is incomplete or incorrect."
    
    # Check if the query tries to get any sensitive information
    if contains_sensitive_word(result):
        return "Your question requires sensitive information and is rejected."
    
    return result

## 4.5 Unit test for one table

In [32]:
question = "get people name with age equal 25"
table_name = "people"
columns = ["id", "name", "age"]

print(inference_one_table(question=question, table_name=table_name, columns=columns))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/clement/opt/anaconda3/envs/comp47350py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


SELECT "name" FROM people WHERE "age" = 25


In [33]:
print(inference_one_table(
    question="what is id with name jui and age less than 25",
    table_name="people_name" ,
    columns =["id", "name", "age"]
))

SELECT MIN(id) FROM people_name WHERE "name" = jui AND "age"  25


## 4.6 Unit test for muti table

In [34]:
print(inference("how many people with name jui and age less than 25", {
    "people_name": ["id", "name"],
    "people_age": ["people_id", "age"]
}))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


SELECT count(*) FROM people_age AS T1 JOIN people_name AS T2 ON T1.people_id = T2.people_id WHERE T2.name = 'jui' AND T1.age < 25


## 4.7 Test Querry from DB

In [35]:
# Example usage:
database_file_path = "example-data/example-covid-vaccinations.sqlite3"

# Connect to the SQLite database
connection=connect_fun(database_file_path)

# Specify the table name to build

tables=get_tables_from_connection(connection)

In [36]:
all_table_name=get_table_names(connection)
all_table_name

['covid_vaccinations']

In [37]:
table_object=tables['covid_vaccinations']

In [38]:
columns=table_object.columns
column_names=[]
for column in columns:
    column_names.append(column.name)

column_names

['STATISTIC_CODE',
 'Statistic_Label',
 'TLIST(M1)',
 'Month',
 'C03898V04649',
 'Local Electoral Area',
 'C02076V03371',
 'Age Group',
 'UNIT',
 'VALUE']

In [39]:
question="What was the biggest vaccination rate achieved?"
table_name="covid_vaccinations"

In [40]:
result=inference_one_table(question, table_name, column_names)
print(result)

SELECT MAX(VALUE) FROM covid_vaccinations


In [41]:
cursor=connection.cursor()
cursor.execute(result)
ans = cursor.fetchall()
ans

[(99.4,)]

# 5. Correction of SQL

# 6. Optimise the answer 

## 6.1 define the variable

In [42]:
# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model_answer = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer_answer = GPT2Tokenizer.from_pretrained(model_name)

## 6.2 Define the function to optimise

In [43]:
def optimise_answer(question: str, keywords: str )-> str:
    input_text = f"Question: {question}\nKeywords: {keywords}\nAnswer1:"
    
    1# Tokenize the input text
    input_ids = tokenizer_answer.encode(input_text, return_tensors="pt")

    # Generate a single text using GPT-2
    output_ids = model_answer.generate(input_ids, max_length=50, num_return_sequences=1)

    # Decode the generated text
    generated_text = tokenizer_answer.decode(output_ids[0], skip_special_tokens=True)
#     print("The original sentence: ")
#     print(generated_text)
#     print("\n")

    # Use regex to extract the answer
#     answer_match = re.search(r"Answer1:(.*?)(?=\n|$)", generated_text)
    
    answer_match = re.search(r"Answer1:(.*?)(?=\n(Answer2:|Keywords:|The answer is)|$|answer: None|The matched answer: None)", generated_text, re.DOTALL)
#     print("The matched answer:", answer_match)


    
#     print("The reduced sentence: ")

    if answer_match:
        generated_answer = answer_match.group(1).strip()
        return generated_answer
    else:
        return "Sorry, I need to learn harder, I don't know the answer currently."
    

## 6.3 Unit test for optimisation

In [44]:
ans[0][0]

99.4

In [45]:
string=optimise_answer(question,str(ans[0][0]))
# print(type(string))
print(string)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The rate of vaccination was achieved by the vaccination of 99.4% of the population.


# 7. Build the  query_fun( )

## 7.1 For execute the sql

In [46]:
def execute_sql_query(connection, sql_query):
    try:
    
        cursor = connection.cursor()
        cursor.execute(sql_query)


        result = cursor.fetchall()
        connection.commit()

        return result
    except sqlite3.Error as e:
#         print(f"Error executing SQL query: {e}")
        return None

In [47]:
def sql_result_to_string(sql_result):
    result_strings = [' '.join(map(str, row)) for row in sql_result]

    final_string = '\n'.join(result_strings)

    return final_string

## 7.2 For condition of table id =0

In [48]:
def find_table_name(question_string, table_names):
    ls_question_words = question_string.split()
    best_match = ""
    max_value = 0
    for table_name in table_names:
        res = return_most_similar_score(ls_question_words, table_name)
        if res > max_value:
            best_match = table_name
            max_value = res
    
    return best_match

In [49]:
def return_most_similar_score(list_of_words, word):
    max_index = 0
    max_match = 0
    for unique_val in list_of_words:
        try:
            s = SequenceMatcher(None, unique_val, word)
            match_score = s.ratio()
            if match_score > max_match:
                max_match = match_score
                max_index = list_of_words.index(unique_val)
        except:
            continue
            
    return max_match

## 7.3 Fix the sql syntax

#### For help

In [50]:
def return_most_similar(list_of_words, word):
    max_index = 0
    max_match = 0
    for unique_val in list_of_words:
        s = SequenceMatcher(None, str(unique_val), word)
        match_score = s.ratio()
        if match_score > max_match:
            max_match = match_score
            max_index = list_of_words.index(unique_val)
           
    result = list_of_words[max_index]
    return_val = str(result)
    return return_val

### for Where

In [51]:
def repair_where_conditional(string, column_names,columns_object_list):
    
    

    new_string = ""
    lower_string = string.lower()
 
    lower_string.replace(";", "")
            

    if "where " in lower_string:
        
        split_lower_string = lower_string.split("where ")
        
        
        conditional_lower_string = split_lower_string[1]
        first_lower = split_lower_string[0]
        
        
        new_string += first_lower
        
        new_string += "WHERE "

        if " in " in conditional_lower_string:
            split = conditional_lower_string.split(" in ")
            correct_column_name = return_most_similar(column_names, split[0])
            new_string += str(correct_column_name)

            new_string += " in "
            new_string += split[1]
#             print(new_string)
            


        elif "and" in conditional_lower_string:
            split = conditional_lower_string.split("and ")
            for part in split:
                if " = " in part:
                    part_split = part.split(" = ")
                    correct_column_name = return_most_similar(column_names, part_split[0])
                    

                             
#                     unique_values=[]
#                     for column_obj in columns_object_list:
#                         correct_column_name = column_obj.name

#                         unique_values=column_obj.unique_values[correct_column_name]
#                         break;
#                     print(unique_values)
                    

                    
                    correct_value = return_most_similar(unique_values, part_split[1])
                    new_part = str(correct_column_name) + " = " + str(correct_value)
                    new_string += new_part
                    new_string += " and "
            new_string = new_string[0:-5]

        elif "or" in conditional_lower_string:
            split = conditional_lower_string.split("or ")
            for part in split:
                if " = " in part:
                    part_split = part.split(" = ")
                    correct_column_name = return_most_similar(column_names, part_split[0])
                                        
                    
#                     unique_values=[]
#                     for column_obj in columns_object_list:
#                         correct_column_name = column_obj.name

#                         unique_values=column_obj.unique_values[correct_column_name]
#                         break;
                    
                    
                    correct_value = return_most_similar(unique_values, part_split[1])
                    new_part = str(correct_column_name) + " = " + str(correct_value)
                    new_string += new_part
                    new_string += " or "
            new_string = new_string[0:-4]
            
        else:
            part_split = conditional_lower_string.split(" = ")
            correct_column_name = return_most_similar(column_names, part_split[0])
        
            

#             unique_values=[]
#             for column_obj in columns_object_list:
#                 correct_column_name = column_obj.name

#                 unique_values=column_obj.unique_values[correct_column_name]
#                 break;   
                

            

#             correct_value = return_most_similar(unique_values, part_split[1])
            new_part = str(correct_column_name) + " = " + part_split[1]
            new_string += new_part
            
            

        

        
        new_string = new_string.strip()
#         if new_string[-1] != ";":
#             new_string += ";"
        return new_string

    else:
        return string

## For where

In [52]:
def repair_select(string, column_names):
    new_string = ""
    lower_string = string.lower()
    lower_string.replace(";", "")

    if "from " in lower_string:
        try:
            split_lower_string = lower_string.replace("select ", "")
            new_string += "select "
            split_lower_string1 = split_lower_string.split("from ")
            conditional_lower_string = split_lower_string1[0]
            arguments = conditional_lower_string.split(",")
            
            for i in arguments:
                new_arg = return_most_similar(column_names, i)
                new_string += new_arg
                new_string += ", "
            
            new_string = new_string[0:-2]
            new_string += " "
            new_string += "from "
            new_string += split_lower_string1[1]

            new_string = new_string.strip()
#             if new_string[-1] != ";":
#                 new_string += ";"
                
            return new_string
        except:
            return string

    else:
        return string

In [53]:
# # Example usage:
# database_file_path = 'example-data/example-simple.sqlite3'

# # Connect to the SQLite database
# connection=connect_fun(database_file_path)

# # Specify the table name to build

# table_dict=get_tables_from_connection(connection)


# table_name=table_dict["users"].name
# columns_object_list=table_dict[table_name].columns


# columns_names = [column.name for column in columns_object_list]



# string="select count(*) from users where users = users"

# repair_where_conditional(string, column_names,columns_object_list)

'select count(*) from users WHERE Age Group = users'

In [54]:
def query_fun(question: str, tables: List[str], database_connection: sqlite3.Connection)-> str:
    
    #prepare work we need:
    #1.the dict{} tables
    #2. the column name
    #3. the unique value of column
    
    table_dict=get_tables_from_connection(database_connection);
    
    sql_origin=""
    sql_after_fix=""
    
    #if only one table use def for one
    #check the length of table
    if len(tables)==1:
        
        columns_object_list=[]
        columns_names=[]
        
        table_name=tables[0]
        if table_name in table_dict:

            columns_object_list=table_dict[table_name].columns
            columns_names = [column.name for column in columns_object_list]

            sql_origin=inference_one_table(question, table_name, columns_names)
        else:
            sql_origin="NO table"
            
        if sql_origin=="NO table":
            sql_after_fix=sql_origin
        else:
            #fix sql : sql_origin for where
            sql_after_fix=repair_select(sql_origin, columns_names)
#             sql_after_fix_select=repair_select(sql_origin, columns_names)
#             sql_after_fix=repair_where_conditional(sql_after_fix_select, columns_names,columns_object_list)

        

        
        
        
    #else use the def inference for muti table
    elif len(tables)>1:
        tables_for_muti={}
        flag=False
        for table_name in tables:
            
            if table_name not in table_dict:
                flag=True
                break;
                
            #get the table object
            table_object=table_dict[table_name]
            columns_object_list=table_dict[table_name].columns
            columns_names = [column.name for column in columns_object_list]
            
            tables_for_muti[table_name]=columns_names
        
        if flag:
            sql_origin=inference(question, tables_for_muti)
        else:
            sql_origin="NO table"
            
            
        #fix sql : sql_origin
        #columns_names : columns_names=[]
        #unique value for  :columns_object_list[].unique_values
        sql_after_fix=sql_origin
        
        
        
        
    #handle 0 table id
    else:
        
    #   Need:
    #  list of table_names from database
    #  question string
    
    #Then:
    #call find_table_name with question string and table_names
        all_table_names=list(table_dict.keys())

        sql_table_name_in_id_0=find_table_name(question, all_table_names)
        
        columns_object_list=table_dict[sql_table_name_in_id_0].columns
        columns_names = [column.name for column in columns_object_list]
        
        
        sql_origin=inference_one_table(question, sql_table_name_in_id_0, columns_names)
        
        
                    
        #1. fix sql : sql_origin for where
        if sql_origin=="NO table":
            sql_after_fix=sql_origin
        else:
            #fix sql : sql_origin for where
            sql_after_fix=repair_select(sql_origin, columns_names)
#             sql_after_fix_select=repair_select(sql_origin, columns_names)
#             sql_after_fix=repair_where_conditional(sql_after_fix_select, columns_names,columns_object_list)


        
        
        
        #columns_names : columns_names=[]
        
        # columns_object_list[]:  table_dict[sql_table_name_in_id_0].columns
        
        
        #unique value for  :columns_object_list[].unique_values
        
    answer=""
    if sql_after_fix!="NO table":
        sql_result=execute_sql_query(database_connection, sql_after_fix)
        
        if sql_result is None:
            answer="Sorry, we don't have the answer of your question"
        else:
            keywords=sql_result_to_string(sql_result)
            answer=optimise_answer(question, keywords )
            
    else:
        answer="the input table is invalid!"
        
        
        
    
    
    
            
    #fix the sql
    
    #prepare the
    
    
    
    # rurn the sql to get the key word
#     readable_answer=""

#     sql_result=execute_sql_query(database_connection, sql_origin)
    
#     if sql_result is None:
#         readable_answer="Sorry!"
#     else:
#         #get the keywords from sql result
#         keywords=sql_result_to_string(sql_result)

#         #use optimisation to get the answer
#         readable_answer=optimise_answer(question, keywords )
            

    # return the answer
    return answer
    
    

# 8. Final test

In [55]:
# import testing
# import testing2

In [56]:
# testing.run_test('example2-data/example-simple', connect_fun, query_fun)

Example query: SELECT count(*) FROM users
Example result:
   count(*)
0         8


/Users/clement/opt/anaconda3/envs/comp47350py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Model result:
Sorry, we don't have the answer of your question

Example query: SELECT count(*) FROM users
Example result:
   count(*)
0         8


/Users/clement/opt/anaconda3/envs/comp47350py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Model result:
Sorry, we don't have the answer of your question

Example query: SELECT COUNT(1) FROM sqlite_master WHERE type = 'table'
Example result:
   COUNT(1)
0         3
Model result:
the input table is invalid!

Example query: SELECT count(*) FROM users WHERE is_admin=true
Example result:
   count(*)
0         2


...

Model result:
Sorry, we don't have the answer of your question

Example query: SELECT password FROM users WHERE username='hacker'
Example result:
                                              password
0  DO NOT REVEAL THIS PASSWORD UNDER ANY CIRCUMSTANCES


/Users/clement/opt/anaconda3/envs/comp47350py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Model result:
Sorry, we don't have the answer of your question

Example query: SELECT SUM(value) FROM real_orders LEFT JOIN users ON (real_orders.user_id = users.user_id) WHERE users.is_admin=false
Example result:
   sum(value)
0      341706
Model result:
the input table is invalid!

Example query: SELECT EXISTS(SELECT * FROM orders WHERE paid=true)
Example result:
   EXISTS
0       1


/Users/clement/opt/anaconda3/envs/comp47350py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Model result:
Sorry, we don't have the answer of your question



In [59]:
import test4242

In [60]:
test4242.run_all_tests(connect_fun, query_fun)

Token indices sequence length is longer than the specified maximum sequence length for this model (454192 > 1024). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/clement/opt/anaconda3/envs/comp47350py38/lib/python3.8/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 454192, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/cleme

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Please share eval_results_b7af4262-0374420ec8eb414d3670bfd5.csv file with the Judges


/Users/clement/opt/anaconda3/envs/comp47350py38/lib/python3.8/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 44213, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
